In [1]:
cd ..

/home/mehdi2277/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research


In [2]:
import torch.optim as optim
import torch.nn as nn

from tree_to_sequence.program_datasets import TreeANCDataset
from tree_to_sequence.tree_to_sequence_attention_anc import TreeToSequenceAttentionANC
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.sequence_encoder import SequenceEncoder
from tree_to_sequence.multilayer_lstm_cell import MultilayerLSTMCell
from tree_to_sequence.translating_trees import tree_to_list, decode_tokens, print_tree
from neural_nets_library import training
from ANC.util import printProgram

In [3]:
num_vars = 10
num_ints = 11

for_ops = {
    "Var": 0,
    "Const": 1,
    "Plus": 2,
    "Minus": 3,
    "EqualFor": 4,
    "LeFor": 5,
    "GeFor": 6,
    "Assign": 7,
    "If": 8,
    "Seq": 9,
    "For": 10
}

for_ops = {"<" + k.upper() + ">": v for k,v in for_ops.items()}

lambda_calculus_ops = {
                "<VARIABLE>": 0,
                "<ABSTRACTION>": 1,
                "<NUMBER>": 2,
                "<BOOLEAN>": 3,
                "<NIL>": 4,
                "<IF>": 5,
                "<CONS>": 6,
                "<MATCH>": 7,
                "<UNARYOPER>": 8,
                "<BINARYOPER>": 9,
                "<LET>": 10,
                "<LETREC>": 11,
                "<TRUE>": 12,
                "<FALSE>": 13,
                "<TINT>": 14,
                "<TBOOL>": 15,
                "<TINTLIST>": 16,
                "<TFUN>": 17,
                "<ARGUMENT>": 18,
                "<NEG>": 19,
                "<NOT>": 20,
                "<PLUS>": 21,
                "<MINUS>": 22,
                "<TIMES>": 23,
                "<DIVIDE>": 24,
                "<AND>": 25,
                "<OR>": 26,
                "<EQUAL>": 27,
                "<LESS>": 28,
                "<APPLICATION>": 29,
                "<HEAD>": 30,
                "<TAIL>": 31
            }

In [4]:
input_eos_token = False
input_as_seq = False
use_embedding = True
eos_bonus = 1 if input_eos_token and input_as_seq else 0
long_base_case = True
binarize = True

In [5]:
is_lambda_calculus = False
M, R = 5, 3
N = 11

for_anc_dset = TreeANCDataset("ANC/Easy-arbitraryForListWithOutput.json", 
                              is_lambda_calculus, num_ints = M,
                              binarize=binarize, input_eos_token=input_eos_token, 
                              use_embedding=use_embedding, long_base_case=long_base_case, 
                              input_as_seq=input_as_seq, cuda=True)

In [6]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform(param, -stdev, stdev)

In [7]:
embedding_size = 30
hidden_size = 30
num_layers = 1
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops.keys()) + eos_bonus

if input_as_seq:
    encoder = SequenceEncoder(encoder_input_size, hidden_size, num_layers, attention=True, use_embedding=use_embedding)
else:
    encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2], attention=True, use_embedding=use_embedding)

decoder = MultilayerLSTMCell(N + 3*R + hidden_size, hidden_size, num_layers)
program_model = TreeToSequenceAttentionANC(encoder, decoder, hidden_size, embedding_size, M, R, 
                                           alignment_size=alignment_size, align_type=align_type,
                                           correctness_weight=1, halting_weight=0, confidence_weight=0, 
                                           efficiency_weight=0)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [8]:
program_model = program_model.cuda()

In [9]:
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=100, factor=0.8)

In [10]:
import torch
program_model.load_state_dict(torch.load('program_model.pth'))

In [19]:
training.train_model_tree_to_anc(program_model, for_anc_dset, 
                                 optimizer, lr_scheduler=lr_scheduler, num_epochs=1, 
                                 batch_size=100, plateau_lr=True, print_every=200)

Epoch 0/0
----------
Epoch Number: 0, Batch Number: 200, Training Loss: 5.6003
Time so far is 0m 35s
Epoch Number: 0, Batch Number: 400, Training Loss: 5.4846
Time so far is 1m 9s
Epoch Number: 0, Batch Number: 600, Training Loss: 6.2591
Time so far is 1m 43s
Epoch Number: 0, Batch Number: 800, Training Loss: 5.5653
Time so far is 2m 17s
Epoch Number: 0, Batch Number: 1000, Training Loss: 5.3003
Time so far is 2m 51s
Epoch Number: 0, Batch Number: 1200, Training Loss: 5.6930
Time so far is 3m 25s
Epoch Number: 0, Batch Number: 1400, Training Loss: 5.4927
Time so far is 3m 59s
Epoch Number: 0, Batch Number: 1600, Training Loss: 5.0219
Time so far is 4m 32s
Epoch Number: 0, Batch Number: 1800, Training Loss: 5.2881
Time so far is 5m 7s
Epoch Number: 0, Batch Number: 2000, Training Loss: 5.4069
Time so far is 5m 41s
Epoch Number: 0, Batch Number: 2200, Training Loss: 5.2302
Time so far is 6m 15s
Epoch Number: 0, Batch Number: 2400, Training Loss: 5.2466
Time so far is 6m 50s
Epoch Number:

KeyboardInterrupt: 

In [ ]:
# i = 0

# for prog, target in for_anc_dset:
# #     controller = program_model.forward_prediction(prog)
# #     util.printProgram(controller, 0.5)
    
# #     loss.backward()
    
# #     for name, param in program_model.named_parameters():
# #         print(name)
# #         print(param.grad)
        
# #     optimizer.step()
# #     optimizer.zero_grad()

#     if i == 3:
#         break
        
#     i += 1

In [33]:
for prog, target in for_anc_dset[0:10]:
    print(decode_tokens(tree_to_list(prog), 10, M, for_ops))
    controller = program_model.forward_prediction(prog)
    
    input_memory = target[0][0]
    correct_memory = target[1][0]
    prediction_memory, _ = controller.forward_prediction([input_memory])
    
    # printProgram(controller, 0.5)
    print(correct_memory[0])
    print(prediction_memory[0])
    
    mem_diff = correct_memory[0] - prediction_memory[0].data
    print(torch.sum(mem_diff * mem_diff))

['<ASSIGN>', '<VAR>', 'a0', '<CONST>', 2]

 0
 0
 1
 0
 0
[torch.cuda.FloatTensor of size 5 (GPU 0)]

Variable containing:
 0.4155
 0.1461
 0.1461
 0.1461
 0.1461
[torch.cuda.FloatTensor of size 5 (GPU 0)]

0.965798556804657
['<ASSIGN>', '<VAR>', 'a0', '<MINUS>', '<CONST>', 1, '<VAR>', 'a0']

 0
 1
 0
 0
 0
[torch.cuda.FloatTensor of size 5 (GPU 0)]

Variable containing:
 0.4155
 0.1461
 0.1461
 0.1461
 0.1461
[torch.cuda.FloatTensor of size 5 (GPU 0)]

0.965796947479248
['<ASSIGN>', '<VAR>', 'a0', '<CONST>', 0]

 1
 0
 0
 0
 0
[torch.cuda.FloatTensor of size 5 (GPU 0)]

Variable containing:
 0.4155
 0.1461
 0.1461
 0.1461
 0.1461
[torch.cuda.FloatTensor of size 5 (GPU 0)]

0.4270246624946594
['<ASSIGN>', '<VAR>', 'a0', '<VAR>', 'a0']

 1
 0
 0
 0
 0
[torch.cuda.FloatTensor of size 5 (GPU 0)]

Variable containing:
 0.9982
 0.0004
 0.0004
 0.0004
 0.0004
[torch.cuda.FloatTensor of size 5 (GPU 0)]

3.892137556249509e-06
['<ASSIGN>', '<VAR>', 'a0', '<PLUS>', '<CONST>', 3, '<VAR>', 'a0']

